<a href="https://colab.research.google.com/github/wendy60/Hybrid-recommender-system/blob/second-submit/SLi_REC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I use an open source project from github, so I need to declare the copyright for each model. I use the MIND public dataset and the python package -- recommenders from microsoft.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# **Global Settings and Imports**

In [ ]:
pip install recommenders

     |████████████████████████████████| 314 kB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 38.2 MB/s 
     |████████████████████████████████| 310 kB 48.5 MB/s 
     |████████████████████████████████| 1.5 MB 62.9 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 636 kB 47.5 MB/s 
     |████████████████████████████████| 11.8 MB 13.8 MB/s 
     |████████████████████████████████| 12.4 MB 42.2 MB/s 
     |████████████████████████████████| 61 kB 6.6 MB/s 
     |████████████████████████████████| 59 kB 5.2 MB/s 
     |████████████████████████████████| 22.3 MB 58.5 MB/s 
     |████████████████████████████████| 749 kB 52.7 MB/s 
     |████████████████████████████████| 59 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 49.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705355 sha256=093e7908b420597961dd630f4bcdaf915b7cd8

In [ ]:
pip install tensorflow-gpu==1.15.2

     |████████████████████████████████| 410.9 MB 29 kB/s 
     |████████████████████████████████| 503 kB 65.9 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 3.8 MB 38.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9c27cd35ecbc708c633bde795e0c6f4637ea4d1bc3b985f810ca6277bc3ca839
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
import re
import sys
import os
import logging
#import papermill as pm
#import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
#from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download

from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


# **Input dataset**

Firstly，I need to jump into the correct file path to access the data file in my google drive

In [ ]:
#link google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#enter the folder
!ls
import os
os.chdir('/content/drive/My Drive/graduation- project/recommenders-main')
!ls

drive  sample_data
AUTHORS.md	    data	 LICENSE	 recommenders  tests
CODE_OF_CONDUCT.md  data1	 MANIFEST.in	 scenarios     tools
conda.md	    docs	 NEWS.md	 SECURITY.md   tox.ini
contrib		    examples	 pyproject.toml  SETUP.md      Untitled0.ipynb
CONTRIBUTING.md     GLOSSARY.md  README.md	 setup.py


In [ ]:
##  config file
yaml_file = './recommenders/models/deeprec/config/sli_rec.yaml'

# **Prepare parameters**

In [ ]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join(".", "data1")


In [ ]:
train_num_ngs = 4 # number of negative instances with a positive instance for training


In [ ]:
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')

# **Prepare hyper-parameters**

In [ ]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, 
                          # provides the number of negative instances for each positive instance for loss computation.
           
            )

print(hparams)

kg_file=None,user_clicks=None,FEATURE_COUNT=None,FIELD_COUNT=None,data_format=None,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=./data1/summary/,MODEL_DIR=./data1/model/,wordEmb_file=None,entityEmb_file=None,contextEmb_file=None,news_feature_file=None,user_history_file=None,use_entity=True,use_context=True,doc_size=None,history_size=None,word_size=None,entity_size=None,entity_dim=None,entity_embedding_method=None,transform=None,train_ratio=None,dim=None,layer_sizes=[100, 64],cross_layer_sizes=None,cross_layers=None,activation=['relu', 'relu'],cross_activation=identity,user_dropout=True,dropout=[0.3, 0.3],attention_layer_sizes=None,attention_activation=None,attention_dropout=0.0,model_type=sli_rec,method=classification,load_saved_model=False,load_model_name=None,filter_sizes=None,num_filters=None,mu=None,fast_CIN_d=0,use_Linear_part=False,use_FM_part=False,use_CIN_part=False,use_DNN_part=False,init_method=tno

# **Create data loader**

In [ ]:
input_creator = SequentialIterator

# **Create model**

In [ ]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

In [ ]:
test_file = os.path.join(data_path, r'final')
test_num_ngs = 9

# **Evaluate the model**

In [ ]:
# test_num_ngs is the number of negative lines after each possitive line in the test_file
result = model.run_eval(test_file, num_ngs=test_num_ngs)
print(result)


{'auc': 0.5008, 'logloss': 0.6931, 'mean_mrr': 0.6376, 'ndcg@2': 0.5876, 'ndcg@4': 0.6115, 'ndcg@6': 0.6408, 'group_auc': 0.4984}


In [ ]:
## Use regular expressions

import re
tempo = '1004659200\x021010361600\x021010361600\x021010361600\x021010361600\x021010361600'
tempo = tempo.encode('unicode_escape').decode('utf-8').replace(' ', '')
# Use regular matching of special characters starting with \x
result = re.findall(r'\\x[a-f0-9]{2}', tempo)
for x in result:
	#  Replace special characters found
    tempo = tempo.replace(x, '')
#  Finally decode
tempo = tempo.encode('utf-8').decode('unicode_escape')
print(tempo)

100465920010103616001010361600101036160010103616001010361600
